In [2]:
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
import math
import time
from timeit import default_timer as timer


In [ ]:
#editing matplotlib parameters for real time animation update
%matplotlib notebook
plt.rcParams['animation.html'] = 'jshtml'

In [ ]:
#function to update the system to new position

def draw_system(initial_pt = 0,ax=None):

    
    ax.clear()
    ax.tick_params(top=False, right = False, left = False, bottom = False)
    ax.set_xticks([])
    ax.set_yticks([])    
    ax.set_xlim(0,13)
    ax.set_ylim(0,4.5)
    
    tyre_dist = 1
    tyre_rad = 0.5
    
    initial_pt = initial_pt + 7
    
    #crearing tyres
    c1 = patches.Circle((initial_pt-tyre_dist,tyre_rad),tyre_rad, color = 'red')
    c2 = patches.Circle((initial_pt+tyre_dist,tyre_rad),tyre_rad,color = 'red')
    ax.add_patch(c1)
    ax.add_patch(c2)


    #creating rectangle
    rect_x = (initial_pt-tyre_dist-tyre_rad-0.5)
    rect_y = 2*tyre_rad
    rec = patches.Rectangle((rect_x,rect_y),4,3)
    ax.add_patch(rec)

    #creating spring
    link_len = 1
    n_spr_pt = 15
    spr_ht = 0.5
    spring_x = [0,link_len]
    y_centre = rec.get_y()+rec.get_height()/2
    spring_y = [y_centre,y_centre]



    len_spr = (rec.get_x()-link_len) - (link_len)
    spr_pts_x = np.linspace(0,len_spr, n_spr_pt)+link_len
    spr_pts_y = [1 if i%2 == 0 else -1 for i in range(len(spr_pts_x))]
    spr_pts_y = np.array(spr_pts_y)*spr_ht + y_centre

    spring_x = np.append(spring_x,spr_pts_x[1:-1])
    spring_x = np.append(spring_x,[spr_pts_x[-1],rect_x])
    spring_y = np.append(spring_y,spr_pts_y[1:-1])
    spring_y = np.append(spring_y,[y_centre,y_centre])


    ax.plot(spring_x,spring_y)   
    fig.canvas.draw()
    



In [ ]:
#simulating damped vibration for 10 seconds

#creating figure
fig,axes = plt.subplots(2,1,figsize = (10,6))

#modifying axes
ax = axes[0]
ax.spines[['right', 'top']].set_visible(False)
ax.spines[['left','bottom']].set_linewidth(4)
ax2 = axes[1]


#setting of parameters of the system
t = 0
u_0 = 3
v_0 = 0
zita = 0.1
f = 0.5
wo = 2*math.pi*f
w = wo/math.sqrt(1-zita**2)


#initial displacement
for i in np.linspace(10,10+u_0,30):
    draw_system(i,ax)
    time.sleep(0.01)


    
timee = []
d = []

while t<10:
    
    u = math.exp(-zita*w*t)*(u_0*math.cos(wo*t)*wo+zita*w*u_0+v_0*math.sin(wo*t))/wo
    
    d = d+[u]
    timee = timee+[t]
    ax2.clear()
    ax2.plot(timee,d)
    ax2.plot(timee[-1],d[-1],'o')
    draw_system(u,ax)
    t = t+0.05
    time.sleep(0.0001)
    

In [3]:
#simulating undamped vibration for 10 seconds



fig,axes = plt.subplots(2,1,figsize = (10,7))
ax = axes[0]
ax.set_xlim(22)
ax2 = axes[1]

t = 0
u_0 = 1
v_0 = 0
f = 10
w = 2*math.pi*f


for i in np.linspace(10,10+u_0,10):
    draw_system(i,ax)
    time.sleep(0.1)

    
timee = []
d = []
while t<10:
    u = 10+u_0*math.cos(w*t) + (v_0/w) * math.sin(w*t)
    d = d+[u]
    timee = timee+[t]
    ax2.clear()
    ax2.plot(timee,d)
    draw_system(u,ax)
    t = t+0.05
    time.sleep(0.0001)

In [ ]:
#creating video

from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

fig,axes = plt.subplots(2,1,figsize = (10,7))
ax = axes[0]
ax2 = axes[1]

ax.spines[['right', 'top']].set_visible(False)
ax.spines[['left','bottom']].set_linewidth(4)


duration = 13
t = 0
u_0 = 2.5
v_0 = 0
zita = 0.07
f = 0.5
wo = 2*math.pi*f
w = wo/math.sqrt(1-zita**2)

def make_frame(tt):
    global t
    if(tt<=2):
        u = u_0*tt/2 
    else:
        t = tt-2
#         u = math.exp(-zita*w*t)*(u_0*math.cos(wo*t)*wo+zita*w*u_0+v_0*math.sin(wo*t))/wo
        u = u_0*math.cos(w*t) + (v_0/w) * math.sin(w*t)
    d1.append(u)
    timee.append(tt)
    ax2.clear()
    ax2.set_ylim(-u_0-0.5,u_0+0.5)
    ax2.set_xlim(0,duration)
    ax2.plot(timee,d1)
    ax2.plot(timee[-1],d1[-1],'o')
    ax2.set_xlabel('time (sec)')
    ax2.set_ylabel('postition from mean')
    draw_system(u,ax)
    fig.suptitle('Undamped SDOF Vibration')
    return mplfig_to_npimage(fig)

d1 = []
timee = []

# creating animation
animation = VideoClip(make_frame, duration = duration)
 
# displaying animation with auto play and looping
animation.ipython_display(fps = 30, loop = True, autoplay = True)